### Test out clipping function with shape file and dataset

In [40]:
import xarray as xr
import geopandas as gpd
import s3fs
#import rasterio as rio
import requests
from shapely.geometry import mapping  

In [41]:
county_shp = gpd.read_file("./tl_2019_us_county.shp")
#print(countyshp)
type(county_shp)

geopandas.geodataframe.GeoDataFrame

In [42]:
county_shp

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,13,123,00351260,13123,Gilmer,Gilmer County,06,H1,G4020,None,None,None,A,1103886891,12283371,+34.6905055,-084.4546244,None
3229,27,135,00659513,27135,Roseau,Roseau County,06,H1,G4020,None,None,None,A,4329471929,16913419,+48.7610683,-095.8215042,None
3230,28,089,00695768,28089,Madison,Madison County,06,H1,G4020,298,27140,None,A,1850602740,71274123,+32.6343703,-090.0341603,None
3231,48,227,01383899,48227,Howard,Howard County,06,H1,G4020,None,13700,None,A,2333039143,8841783,+32.3034298,-101.4387208,None


In [43]:
def begin_s3_direct_access(url: str="https://data.gesdisc.earthdata.nasa.gov/s3credentials"):
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})

fs = begin_s3_direct_access()

type(fs)

s3fs.core.S3FileSystem

In [45]:
merra2 = fs.glob("s3://gesdisc-cumulus-prod-protected/MERRA2/M2T1NXSLV.5.12.4/2000/01/*") #only read merra2 2000/01

merra2_files = merra2[0:1]
len(merra2_files)

1

In [46]:
merra2_Dataset = xr.open_mfdataset(
    paths=[fs.open(f) for f in merra2_files],
    combine='by_coords',
    mask_and_scale=True,
    decode_cf=True,
    #chunks={'latitude': [25,53],   # These were chosen arbitrarily. You must specify 
    #        'longitude': [-125,-67] # chunking that is suitable to the data and target
    #     'time': 100}      # analysis.
)

#print(merra2_Dataset)

In [47]:
temp = merra2_Dataset.T2M
temp

<xarray.DataArray 'T2M' (time: 24, lat: 361, lon: 576)>
dask.array<open_dataset-b4194cafa23674407508267d18dd8186T2M, shape=(24, 361, 576), dtype=float32, chunksize=(24, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * time     (time) datetime64[ns] 2000-01-01T00:30:00 ... 2000-01-01T23:30:00
Attributes:
    long_name:       2-meter_air_temperature
    units:           K
    fmissing_value:  [1.e+15]
    standard_name:   2-meter_air_temperature
    vmax:            [1.e+15]
    vmin:            [-1.e+15]
    valid_range:     [-1.e+15  1.e+15]

In [48]:
temp_daily_mean = temp.resample(time="1D").mean(dim='time', skipna=True)
temp_daily_mean

<xarray.DataArray 'T2M' (time: 1, lat: 361, lon: 576)>
dask.array<copy, shape=(1, 361, 576), dtype=float32, chunksize=(1, 361, 576), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0

In [32]:
def clip_to_shape(ds, gdf):
    """
    Clip xarray.Dataset using GeoPandas DataFrame. (Author: Caleb Spradlin)
    """
    ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    ds.rio.write_crs("epsg:4326", inplace=True)
    clipped_ds = ds.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=False)
    
    min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
    clipped_ds = clipped_ds.sel(lon=slice(min_lon, max_lon), lat=slice(min_lat, max_lat))
    
    return clipped_ds

In [50]:
#clip_to_shape(temp_daily_mean,county_shp)
temp_daily_mean[0].rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
temp_daily_mean[0].rio.write_crs("epsg:4326", inplace=True)
clipped_ds = temp_daily_mean[0].rio.clip(county_shp.geometry.apply(mapping), county_shp.crs, drop=False)

AttributeError: 'NoneType' object has no attribute '__geo_interface__'